In [4]:
!pip install pandas numpy matplotlib seaborn sqlalchemy psycopg2-binary

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine

%matplotlib inline


In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [19]:
df1 = pd.read_csv(r"F:\Business Analytics\Projects\Customer Segmentation & Retention Analytics\transactions.csv")
df2 = pd.read_csv(r"F:\Business Analytics\Projects\Customer Segmentation & Retention Analytics\customers.csv")
df3 = pd.read_csv(r"F:\Business Analytics\Projects\Customer Segmentation & Retention Analytics\churn_labels.csv")

print("Transactions Shape:", df1.shape)
print("Customers Shape:", df2.shape)
print("Churn Shape:", df3.shape)


Transactions Shape: (600000, 4)
Customers Shape: (200000, 4)
Churn Shape: (200000, 2)


In [21]:
df1.head()

,transaction_id,customer_id,transaction_date,amount
0,1,254873,2024-11-12 02:33:33,608.33
1,2,258758,2023-08-02 00:56:20,859.52
2,3,255933,2023-08-08 03:21:13,116.04
3,4,245819,2022-07-13 01:18:02,552.89
4,5,269315,2023-10-24 21:42:22,2075.85


In [28]:
df1.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600000 entries, 0 to 599999
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   transaction_id    600000 non-null  int64         
 1   customer_id       600000 non-null  int64         
 2   transaction_date  600000 non-null  datetime64[ns]
 3   amount            600000 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 18.3 MB


In [29]:
print("Duplicates before:", df1.duplicated().sum())
df1.drop_duplicates(inplace=True)
print("Duplicates after:", df1.duplicated().sum())


Duplicates before: 0
Duplicates after: 0


In [30]:
df1['transaction_date'] = pd.to_datetime(df1['transaction_date'])
df2['signup_date'] = pd.to_datetime(df2['signup_date'])


In [31]:
print("Negative Transactions:", (df1['amount'] <= 0).sum())
df1 = df1[df1['amount'] > 0]


Negative Transactions: 0


In [32]:
df = df1.merge(df2, on='customer_id', how='left')
df = df.merge(df3, on='customer_id', how='left')

print("Final Shape:", df.shape)
df.head()


Final Shape: (600000, 8)


,transaction_id,customer_id,transaction_date,amount,signup_date,region,channel,is_churn
0,1,254873,2024-11-12 02:33:33,608.33,2022-11-14 16:21:36,South,Online,1
1,2,258758,2023-08-02 00:56:20,859.52,2022-01-09 06:41:20,West,Offline,1
2,3,255933,2023-08-08 03:21:13,116.04,2021-08-28 19:46:27,South,Online,0
3,4,245819,2022-07-13 01:18:02,552.89,2021-06-17 06:40:40,West,Offline,1
4,5,269315,2023-10-24 21:42:22,2075.85,2023-09-07 02:48:51,West,Online,0


In [33]:
df.rename(columns={'amount': 'Revenue'}, inplace=True)


In [34]:
snapshot_date = df['transaction_date'].max() + pd.Timedelta(days=1)
snapshot_date


Timestamp('2024-12-31 23:59:09')

In [36]:
rfm = df.groupby('customer_id').agg({
    'transaction_date': lambda x: (snapshot_date - x.max()).days,
    'transaction_id': 'count',
    'Revenue': 'sum',
    'is_churn': 'max'
})

rfm.columns = ['Recency', 'Frequency', 'Monetary', 'Churn_Label']

rfm.head()


,Recency,Frequency,Monetary,Churn_Label
customer_id,,,,
100001,426,3,2984.05,1
100002,375,2,792.21,0
100003,186,3,1971.75,0
100004,521,4,3095.71,0
100005,449,3,2720.66,0


In [37]:
rfm.shape


(190140, 4)

In [38]:
rfm['R'] = pd.qcut(rfm['Recency'], 5, labels=[5,4,3,2,1])
rfm['F'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1,2,3,4,5])
rfm['M'] = pd.qcut(rfm['Monetary'], 5, labels=[1,2,3,4,5])

rfm['RFM_Score'] = (
    rfm['R'].astype(int)*0.5 +
    rfm['F'].astype(int)*0.3 +
    rfm['M'].astype(int)*0.2
)

rfm.head()


,Recency,Frequency,Monetary,Churn_Label,R,F,M,RFM_Score
customer_id,,,,,,,,
100001,426,3,2984.05,1,2,2,3,2.2
100002,375,2,792.21,0,2,1,1,1.5
100003,186,3,1971.75,0,3,2,2,2.5
100004,521,4,3095.71,0,2,4,3,2.8
100005,449,3,2720.66,0,2,2,3,2.2


In [39]:
def segment(row):
    if row['RFM_Score'] >= 4:
        return "Champions"
    elif row['RFM_Score'] >= 3:
        return "Loyal"
    elif row['Recency'] > 120:
        return "At Risk"
    else:
        return "Potential"

rfm['Segment'] = rfm.apply(segment, axis=1)


In [40]:
rfm['Segment'].value_counts(normalize=True)*100


Segment
At Risk      44.222152
Loyal        28.789839
Champions    25.716840
Potential     1.271169
Name: proportion, dtype: float64

In [41]:
pd.crosstab(rfm['Segment'], rfm['Churn_Label'], normalize='index') * 100


Churn_Label,0,1
Segment,,
At Risk,74.796632,25.203368
Champions,74.992842,25.007158
Loyal,74.996803,25.003197
Potential,74.472487,25.527513


In [42]:
df['OrderMonth'] = df['transaction_date'].dt.to_period('M')
df['CohortMonth'] = df.groupby('customer_id')['transaction_date'].transform('min').dt.to_period('M')

df['CohortIndex'] = (
    (df['OrderMonth'].dt.year - df['CohortMonth'].dt.year) * 12 +
    (df['OrderMonth'].dt.month - df['CohortMonth'].dt.month)
)

cohort = df.groupby(['CohortMonth','CohortIndex'])['customer_id'].nunique().reset_index()

cohort_pivot = cohort.pivot(index='CohortMonth',
                            columns='CohortIndex',
                            values='customer_id')

retention = cohort_pivot.divide(cohort_pivot.iloc[:,0], axis=0)

retention.head()


CohortIndex,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
CohortMonth,,,,,,,,,,,,,,,,,,,,,
2022-01,1.0,0.072787,0.085381,0.072602,0.083961,0.084949,0.082541,0.083405,0.078158,0.082973,...,0.081492,0.081059,0.082911,0.079022,0.083158,0.082973,0.078713,0.081492,0.081615,0.083282
2022-02,1.0,0.080210,0.077627,0.081685,0.080579,0.084932,0.079914,0.078660,0.083751,0.076668,...,0.079767,0.081759,0.081021,0.083456,0.079841,0.079767,0.079472,0.079324,0.079619,NaN
2022-03,1.0,0.077936,0.078518,0.082084,0.079610,0.081211,0.078882,0.083467,0.080483,0.079537,...,0.081356,0.079974,0.082448,0.082375,0.073133,0.080119,0.078809,0.079755,NaN,NaN
2022-04,1.0,0.080038,0.076935,0.084891,0.083380,0.079163,0.082107,0.076299,0.081311,0.077492,...,0.082107,0.080118,0.082186,0.078367,0.083459,0.082902,0.077174,NaN,NaN,NaN
2022-05,1.0,0.079217,0.086842,0.080319,0.082437,0.080827,0.076760,0.079387,0.076845,0.077692,...,0.077862,0.083114,0.079810,0.082776,0.072185,0.081844,NaN,NaN,NaN,NaN


In [43]:
final_df = df.merge(rfm, on='customer_id')

final_df.to_csv("Final_Customer_Analytics.csv", index=False)
rfm.to_csv("Customer_Segments.csv", index=False)
retention.to_csv("Cohort_Retention.csv")


In [44]:
final_df.shape
final_df.head()
final_df.isnull().sum()


transaction_id      0
customer_id         0
transaction_date    0
Revenue             0
signup_date         0
region              0
channel             0
is_churn            0
OrderMonth          0
CohortMonth         0
CohortIndex         0
Recency             0
Frequency           0
Monetary            0
Churn_Label         0
R                   0
F                   0
M                   0
RFM_Score           0
Segment             0
dtype: int64

In [45]:
rfm.describe()
rfm['Segment'].value_counts()


Segment
At Risk      84084
Loyal        54741
Champions    48898
Potential     2417
Name: count, dtype: int64

In [46]:
retention.head()
retention.shape


(36, 36)

In [47]:
import os
os.getcwd()


'C:\\Users\\GOD'

In [ ]:
final_df.to_csv(r"C:\Users\GOD\Final_Customer_Analytics.csv", index=False)
rfm.to_csv(r"C:\Users\GOD\Customer_Segments.csv", index=False)
retention.to_csv(r"C:\Users\GOD\Cohort_Retention.csv")
